Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [34]:
NAME = "Falonne KPAMEGAN"
COLLABORATORS = ""

---

# SparkSQL

In this notebook, we introduce SparkSQL, Spark's interface for working with structured data. From Spark 2.0 and forward, this is the preferred way of implementing Spark code, as it contains all of the latest optimisations.

PySpark benefits a lot from SparkSQL, as there is performance parity between Scala, Java, Python and R interfaces for Spark which use the same optimizer. 

---
## Prerequisites

Before running Spark code, we need to start a SparkSession instance. The following block will be common to every notebook so you can run your code.

While your SparkSession is running, you can hit `http://localhost:4040` or `http://host.docker.internal:4040` to get an overview of your Spark local cluster and all operations ongoing.

In [35]:
import os
os.environ["PYSPARK_PYTHON"]="python"
os.environ

environ{'ALLUSERSPROFILE': 'C:\\ProgramData',
        'APPDATA': 'C:\\Users\\fkpamegan\\AppData\\Roaming',
        'CAMLIBS': 'C:\\Program Files\\darktable\\lib\\libgphoto2\\2.5.30',
        'CHOCOLATEYINSTALL': 'C:\\ProgramData\\chocolatey',
        'CHROME_CRASHPAD_PIPE_NAME': '\\\\.\\pipe\\crashpad_1828_UYZHCXBXEQTMZXTD',
        'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files',
        'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files',
        'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files',
        'COMPUTERNAME': 'UL200013001',
        'COMSPEC': 'C:\\windows\\system32\\cmd.exe',
        'CONDA_ALLOW_SOFTLINKS': 'false',
        'CONDA_DEFAULT_ENV': 'pyspark-tutorial',
        'CONDA_EXE': 'C:\\Anaconda3\\Scripts\\conda.exe',
        'CONDA_PREFIX': 'C:\\Anaconda3\\envs\\pyspark-tutorial',
        'CONDA_PROMPT_MODIFIER': '(pyspark-tutorial) ',
        'CONDA_PYTHON_EXE': 'C:\\Anaconda3\\python.exe',
        'CONDA_ROOT': 'C:\\Anaconda3',
        '

In [36]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf().setAppName('lecture-lyon2').setMaster('local')
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

In [37]:
# Import other important libraries

from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql.types import *

import numpy as np
import pandas as pd
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

---
## Part A - On to DataFrames / Datasets

A `Dataset` is a distributed collection of data which provides the benefits of RDDs (strong typing, ability to use lambda functions) with the benefits of SparkSQL's optimized execution engine.

A `DataFrame` is a `Dataset` organized into named columns. It is conceptually equivalent to a table in a relational database, or a data frame in Python/R. Conceptually, a `DataFrame` is a `Dataset` of `Row`s.

As with RDDs, applications can create DataFrames from an existing RDD, a Hive table or from Spark data sources.

# Question

Recall from the previous assignment how we used two tables on students : one for students to grades, another one for students to gender. Let's create a function which takes a RDD of Row and a schema as arguments and generates the corresponding DataFrame.

In [38]:
"""def mean_grade_per_gender(sc, genders, grades):
    joined_rdd = genders.join(grades)
    gender_grades = joined_rdd.map(lambda x: (x[1][0], x[1][1]))
    sum_by_gender = gender_grades.reduceByKey(lambda x, y: x + y)

    count_by_gender = gender_grades.countByKey()

    patate = sum_by_gender.map(lambda gender_to_sum: (gender_to_sum[0] , gender_to_sum[1] / count_by_gender[gender_to_sum[0]])).sortByKey(ascending=False)

    print(patate.collect())

    return patate

    raise NotImplementedError()"""

'def mean_grade_per_gender(sc, genders, grades):\n    joined_rdd = genders.join(grades)\n    gender_grades = joined_rdd.map(lambda x: (x[1][0], x[1][1]))\n    sum_by_gender = gender_grades.reduceByKey(lambda x, y: x + y)\n\n    count_by_gender = gender_grades.countByKey()\n\n    patate = sum_by_gender.map(lambda gender_to_sum: (gender_to_sum[0] , gender_to_sum[1] / count_by_gender[gender_to_sum[0]])).sortByKey(ascending=False)\n\n    print(patate.collect())\n\n    return patate\n\n    raise NotImplementedError()'

In [39]:
"""genders_rdd = spark.sparkContext.parallelize([('1', 'M'), ('2', 'M'), ('3', 'F'), ('4', 'F'), ('5', 'F'), ('6', 'M')])
grades_rdd = spark.sparkContext.parallelize([('1', 5), ('2', 12), ('3', 7), ('4', 18), ('5', 9), ('6', 5)])

result_rdd = mean_grade_per_gender(spark.sparkContext, genders_rdd, grades_rdd)
result_rdd"""

"genders_rdd = spark.sparkContext.parallelize([('1', 'M'), ('2', 'M'), ('3', 'F'), ('4', 'F'), ('5', 'F'), ('6', 'M')])\ngrades_rdd = spark.sparkContext.parallelize([('1', 5), ('2', 12), ('3', 7), ('4', 18), ('5', 9), ('6', 5)])\n\nresult_rdd = mean_grade_per_gender(spark.sparkContext, genders_rdd, grades_rdd)\nresult_rdd"

In [40]:
def create_dataframe(spark, rdd, schema):
    """
    Generate a DataFrame from a RDD of Rows and a schema.
    We assume the RDD is correctly formatted, no need to check for anything.
    """
    # YOUR CODE HERE
    df = spark.createDataFrame(rdd, schema)
    return df
    raise NotImplementedError()

In [41]:
"""
Graded cell

2 points
"""
rdd = spark.sparkContext.parallelize([('1', 'a'), ('2', 'b'), ('3', 'c'), ('4', 'd'), ('5', 'e'), ('6', 'f')])
schema = StructType([StructField('ID', StringType(), True), StructField('letter', StringType(), True)])

result_df = create_dataframe(spark, rdd, schema)
assert result_df.schema == schema
assert result_df.rdd.collect() == rdd.collect()

Let's generate a Dataframe of the students tables for the incoming questions, using our newly created `create_dataframe` function. We also create temporary views for those DataFrames so we can interact with them in SQL.

In [42]:
genders_rdd = spark.sparkContext.parallelize([('1', 'M'), ('2', 'M'), ('3', 'F'), ('4', 'F'), ('5', 'F'), ('6', 'M')])
grades_rdd = spark.sparkContext.parallelize([('1', 5), ('2', 12), ('3', 7), ('4', 18), ('5', 9), ('6', 5)])

genders_schema = StructType([StructField('ID', StringType(), True), StructField('gender', StringType(), True)])
grades_schema = StructType([StructField('ID', StringType(), True), StructField('grade', StringType(), True)])

genders_df = create_dataframe(spark, genders_rdd, genders_schema)
grades_df = create_dataframe(spark, grades_rdd, grades_schema)

genders_df.createOrReplaceTempView('genders')
grades_df.createOrReplaceTempView('grades')

You have two ways of interacting with a Dataframe :

* DataFrames provide a domain-specific language for structured manipulation :

```python
>> genders_df.filter(genders_df['ID'] > 2)
+---+------+
| ID|gender|
+---+------+
|  3|     F|
|  4|     F|
|  5|     F|
|  6|     M|
+---+------+
```

In the more simple cases, you can interact with DataFrames with a syntax close to the Pandas syntax.

```python
>> genders_df[genders_df['ID'] > 2]
+---+------+
| ID|gender|
+---+------+
|  3|     F|
|  4|     F|
|  5|     F|
|  6|     M|
+---+------+
```

* The `sql` function of a SparkSession enables to run SQL queries directly on the frame and returns a DataFrame, on which you can continue your computations

```python
# Register the DataFrame as a SQL temporary view beforehand
>> genders_df.createOrReplaceTempView('genders')
>> spark.sql('SELECT * FROM genders WHERE ID > 2').show()
+---+------+
| ID|gender|
+---+------+
|  3|     F|
|  4|     F|
|  5|     F|
|  6|     M|
+---+------+
```

Don't hesitate to check the [DataFrame Function Reference](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions) for all of the operators you can use on a DataFrame. Use the following cell to experiment :)

In [43]:
# Use this cell to practice your new SQL skills
genders_df[genders_df['ID'] > 2].show()

+---+------+
| ID|gender|
+---+------+
|  3|     F|
|  4|     F|
|  5|     F|
|  6|     M|
+---+------+



# Question

Remember the mean grade per gender question from last assignment ? Remember how unpleasant it was ? Let's do that directly in SparkSQL ! You can do it with whatever way pleases you between programmatic SQL or SparkSQL DSL. 

PS : if you are using programmatic SQL interaction, you may want to define a temporary view of temporary variables. You may want to delete those views at the end of your function with `spark.catalog.dropTempView('your_view')`. Be careful if removing the view, DataFrame are also lazily computed so don't delete your view if you still have not computed and cached the resulting DataFrame.

In [44]:
def mean_grade_per_gender(spark, genders_df, grades_df):
    """
    Given a DataFrame of studentID to grades and studentID to gender, compute mean grade for each gender.
    Assume all studentIDs are present in both DataFrames, making inner join possible, no need to check that.
    Schema of output DataFrame should be gender, mean.
    """
    # Register DataFrames as temporary views
    genders_df.createOrReplaceTempView("genders")
    grades_df.createOrReplaceTempView("grades")
    
    # Perform SQL query to compute the mean grade per gender
    query = """
        SELECT g.gender, AVG(CAST(gr.grade AS DOUBLE)) AS mean
        FROM genders g
        JOIN grades gr
        ON g.ID = gr.ID
        GROUP BY g.gender
    """
    result_df = spark.sql(query)
    
    # Drop the temporary views to clean up
    spark.catalog.dropTempView("genders")
    spark.catalog.dropTempView("grades")
    
    return result_df


In [45]:
"""
Graded cell

3 points
"""
result_df = mean_grade_per_gender(spark, genders_df, grades_df).toPandas()
result_df.columns == ['gender', 'grade']

assert result_df[result_df['gender'] == 'F'].values[0][1] - 11.3 < 0.1
assert result_df[result_df['gender'] == 'M'].values[0][1] - 7.3 < 0.1

---
## Part B - Descriptive statistics in SparkSQL

Let's reload the `FL_insurance_sample.csv` file from last assignment and freely interact with it.

# Question

Load the file by giving a path to the file

In [46]:
def read_csv(spark, path):
    """
    Create a DataFrame by loading an external csv file.
    Assume the file has a header, uses " as double quote and , as delimiter.
    Infer its schema automatically.
    """
    try:
        # Use Spark's read API to load the CSV
        df = spark.read.csv(
            path,
            header=True,           # The file has a header
            inferSchema=True,      # Automatically infer the schema
            quote='"',             # Double quote for escaping
            sep=","                # Comma as the delimiter
        )
        return df
    except Exception as e:
        # In case the file does not exist or any other error, return None
        print(f"Error reading file at {path}: {e}")
        return None

    raise NotImplementedError()

In [47]:
"""
Graded cell

2 points
"""
file_path = 'FL_insurance_sample.csv'
result_df = read_csv(spark, file_path)

assert result_df.schema == StructType([
    StructField('policyID',IntegerType(),True),
    StructField('statecode',StringType(),True),
    StructField('county',StringType(),True),
    StructField('eq_site_limit',DoubleType(),True),
    StructField('hu_site_limit',DoubleType(),True),
    StructField('fl_site_limit',DoubleType(),True),
    StructField('fr_site_limit',DoubleType(),True),
    StructField('tiv_2011',DoubleType(),True),
    StructField('tiv_2012',DoubleType(),True),
    StructField('eq_site_deductible',DoubleType(),True),
    StructField('hu_site_deductible',DoubleType(),True),
    StructField('fl_site_deductible',DoubleType(),True),
    StructField('fr_site_deductible',IntegerType(),True),
    StructField('point_latitude',DoubleType(),True),
    StructField('point_longitude',DoubleType(),True),
    StructField('line',StringType(),True),
    StructField('construction',StringType(),True),
    StructField('point_granularity',IntegerType(),True)
])

# Question

Let's plot the number of different counties in a histogram, like in the previous assignment. We have imported the `bokeh` module for interactive plotting. To do that, return a Pandas a dataframe which contains, for each county, the number of its occurences in the dataset.

_Hint: a Spark Dataframe is distributed on a number of workers, so it cannot be plotted as is. You will need to collect the data you want to plot back in the driver. The `toPandas` is usable to retrieve a Pandas local Dataframe, be careful to only use it on small Dataframes !_

In [48]:
insurance_df = read_csv(spark, 'FL_insurance_sample.csv')
insurance_df.createOrReplaceTempView('insurance')

In [49]:
def count_county(spark, insurance_df):
    """
    Return a Pandas DataFrame containing, for each county, the number of its occurrences in the dataset.
    Schema of the DataFrame should be ['county', 'count'].
    """
    # Group by 'county' and count occurrences
    county_counts = (
        insurance_df.groupBy("county")
        .count()  # Count occurrences for each county
        .withColumnRenamed("count", "count")  
    )
    
    pandas_df = county_counts.toPandas()
    
    return pandas_df

In [50]:
"""
Graded cell

3 points
"""
df = count_county(spark, insurance_df)
result = df.set_index('county').to_dict()['count']

assert result.get('CLAY COUNTY') == 346

In [51]:
data = count_county(spark, insurance_df)
data

,county,count
0,CALHOUN COUNTY,68
1,CLAY COUNTY,346
2,PUTNAM COUNTY,245
3,OKALOOSA COUNTY,1115
4,SUMTER COUNTY,9
5,SUWANNEE COUNTY,154
6,GULF COUNTY,72
7,MADISON COUNTY,81
8,COLUMBIA COUNTY,124
9,ST JOHNS COUNTY,657


In [52]:
# Plot it for fun with bokeh.
data = count_county(spark, insurance_df)

source = ColumnDataSource(data)

hover = HoverTool(tooltips=[
    ("type", "@county"),
    ("count", "@count"),
])

p = figure(x_range=data['county'].values, height=250, title="County counts", tools=[hover])

p.vbar(x='county', top='count', width=0.9, source=source)

p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)

# Postrequisites

In [53]:
spark.stop()